Small improvement over before!

<a id="toc"></a>

# Table of Contents
1.) [Imports](#imports)  
2.) [Load data](#load)  
3.) [Merge Bureau and Bureau_balance](#merge_bureau)  

<a id="imports"></a>

# Imports

In [1]:
import pandas as pd
import numpy as np
import time

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set()

import warnings
warnings.filterwarnings('ignore')

<a id="load"></a>

# Load data

In [2]:
DATA_PATH = "../../data/home_default/"

train = pd.read_csv(DATA_PATH + "train.csv")
test  = pd.read_csv(DATA_PATH + "test.csv")

test['is_train'] = 0
train['is_train'] = 1

print("Shape of train:", train.shape)
print("Shape of test:",  test.shape)

Shape of train: (307511, 123)
Shape of test: (48744, 122)


### Load other data sources

In [6]:
bureau = pd.read_csv(DATA_PATH + "bureau.csv")
bureau_balance = pd.read_csv(DATA_PATH + "bureau_balance.csv")
previous_application = pd.read_csv(DATA_PATH + "previous_application.csv")

<a id="merge_bureau"></a>

# Merge bureau and bureau_balance

### Setup bureau balance - get dummies

In [7]:
# function to create dummy variables of categorical features
def get_dummies(df, cats):
    for col in cats:
        df = pd.concat([df, pd.get_dummies(df[col], prefix=col)], axis=1)
    return df 

bureau_balance = get_dummies(bureau_balance, ["STATUS"])

### Merge bureau and bureau_balance

In [8]:
bureau_balance = bureau_balance.drop(["MONTHS_BALANCE", "STATUS"], axis=1)

# prep for merge
bureau_balance = bureau_balance.groupby("SK_ID_BUREAU").sum()

# Merge
bureau = bureau.merge(right=bureau_balance.reset_index(), how='left', on='SK_ID_BUREAU')

,STATUS_0,STATUS_1,STATUS_2,STATUS_3,STATUS_4,STATUS_5,STATUS_C,STATUS_X
SK_ID_BUREAU,,,,,,,,
5001709,0,0,0,0,0,0,86,11
5001710,5,0,0,0,0,0,48,30
5001711,3,0,0,0,0,0,0,1
5001712,10,0,0,0,0,0,9,0
5001713,0,0,0,0,0,0,0,22


### Fill in new missing values with -1

There won't be any effect if we replace NaN with -1

In [17]:
bureau[["STATUS_0", "STATUS_1", "STATUS_2", "STATUS_3", "STATUS_4", "STATUS_5", "STATUS_C", "STATUS_X"]] = (
    bureau[["STATUS_0", "STATUS_1", "STATUS_2", "STATUS_3", "STATUS_4", "STATUS_5", "STATUS_C", "STATUS_X"]]
        .fillna(-1))

SK_ID_CURR                      0
SK_ID_BUREAU                    0
CREDIT_ACTIVE                   0
CREDIT_CURRENCY                 0
DAYS_CREDIT                     0
CREDIT_DAY_OVERDUE              0
DAYS_CREDIT_ENDDATE        105553
DAYS_ENDDATE_FACT          633653
AMT_CREDIT_MAX_OVERDUE    1124488
CNT_CREDIT_PROLONG              0
AMT_CREDIT_SUM                 13
AMT_CREDIT_SUM_DEBT        257669
AMT_CREDIT_SUM_LIMIT       591780
AMT_CREDIT_SUM_OVERDUE          0
CREDIT_TYPE                     0
DAYS_CREDIT_UPDATE              0
AMT_ANNUITY               1226791
STATUS_0                        0
STATUS_1                        0
STATUS_2                        0
STATUS_3                        0
STATUS_4                        0
STATUS_5                        0
STATUS_C                        0
STATUS_X                        0
dtype: int64

### Split again into predictors, target, and id

In [18]:
train_y = train.TARGET
train_x = train.drop(["TARGET"], axis=1)

test_id = test.SK_ID_CURR
test_x  = test

### Merge train and test data

In [19]:
full = pd.concat([train_x, test_x])
train_N = len(train_x)

### Fill in missing values - Categorical

In [20]:
def fillna_cat(df):
    for col in [col for col in df if df[col].dtype==object]:
        df[col] = df[col].fillna(df[col].mode()[0])
    return df

full                 = fillna_cat(full)
bureau               = fillna_cat(bureau)
previous_application = fillna_cat(previous_application)

### Fill in missing values - Numeric

In [21]:
def fillna_num(df):
    missing_cols = [col for col in df.columns if any(df[col].isnull())]
    for col in missing_cols:
        df[col] = df[col].fillna(df[col].median())
    return df

full                 = fillna_num(full)
bureau               = fillna_num(bureau)
previous_application = fillna_num(previous_application)

### Turn categorical features to dummy columns

In [22]:
previous_application = pd.get_dummies(previous_application)
bureau = pd.get_dummies(bureau)

### Factorize

In [23]:
def factorize_df(df, cats):
    for col in cats:
        df[col], _ = pd.factorize(df[col])
    return df 

# Get categorical features
data_cats = [col for col in full.columns if full[col].dtype == 'object']

# Factorize the dataframe
full = factorize_df(full, data_cats)

### Aggregate Previous Applications Data and Merge with Original Data

[sban](https://www.kaggle.com/shivamb) provided the code ([link](https://www.kaggle.com/shivamb/homecreditrisk-extensive-eda-baseline-model))

In [24]:
# count the number of previous applications for a given ID
prev_apps_count = previous_application[['SK_ID_CURR', 'SK_ID_PREV']].groupby('SK_ID_CURR').count()
previous_application['SK_ID_PREV'] = previous_application['SK_ID_CURR'].map(prev_apps_count['SK_ID_PREV'])

# Average values for all other features in previous applications
prev_apps_avg = previous_application.groupby('SK_ID_CURR').mean()
prev_apps_avg.columns = ['p_' + col for col in prev_apps_avg.columns]
full = full.merge(right=prev_apps_avg.reset_index(), how='left', on='SK_ID_CURR')

### Aggregate Bureau Data and Merge with Original Data

[sban](https://www.kaggle.com/shivamb) provided the code ([link](https://www.kaggle.com/shivamb/homecreditrisk-extensive-eda-baseline-model))

In [25]:
# Average Values for all bureau features 
bureau_avg = bureau.groupby('SK_ID_CURR').mean()
bureau_avg['buro_count'] = bureau[['SK_ID_BUREAU','SK_ID_CURR']].groupby('SK_ID_CURR').count()['SK_ID_BUREAU']
bureau_avg.columns = ['b_' + f_ for f_ in bureau_avg.columns]
full = full.merge(right=bureau_avg.reset_index(), how='left', on='SK_ID_CURR')

### I notice NaN values sneak into full after merging

In [30]:
full = fillna_num(full)

### Split full back into train and test

In [31]:
train_x = full[:train_N]
test_x = full[train_N:]
del full, train_N

### Model imports

In [32]:
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression, Lasso, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, KFold

kfold = KFold(n_splits=2)
    
def rmsle_cv(model):
    rmse= np.sqrt(-cross_val_score(model, train_x, train_y, cv=kfold, scoring="neg_mean_squared_error"))
    print(rmse)
    print()
    print(sum(rmse) / len(rmse))

## LGBM Regressor

(Training takes 545 seconds)

(Scoring takes 722 seconds)

(open Markdown for notes on [LGBM](http://lightgbm.readthedocs.io/en/latest/Python-API.html#lightgbm.LGBMRegressor) and parameter search attempts)

<div hidden>

# LGBM Parameters

boosting_type='gbdt', num_leaves=31, max_depth=-1, learning_rate=0.1, n_estimators=100, subsample_for_bin=200000, objective=None, class_weight=None, min_split_gain=0.0, min_child_weight=0.001, min_child_samples=20, subsample=1.0, subsample_freq=0, colsample_bytree=1.0, reg_alpha=0.0, reg_lambda=0.0, random_state=None, n_jobs=-1, silent=True, **kwargs

<\div>

In [14]:
start = time.time()
lgbm_model = LGBMRegressor(
                            learning_rate= 0.01,
                            num_leaves= 48,
                            num_iteration= 5000,
                            max_depth= 7
                          )
lgbm_model.fit(train_x, train_y)
print("Training took {} seconds".format(round(time.time() - start)))

Training took 933 seconds


### Scoring

In [ ]:
start = time.time()
rmsle_cv(lgbm_model)
print("\nScoring took {} seconds".format(round(time.time() - start)))

## Linear Regressor

(Training takes 11 seconds)

(Scoring takes 13 seconds)

(open Markdown for notes on [Linear Regression](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) and parameter search attempts)

<div hidden>

# Linear Parameters

fit_intercept=True,
normalize=False,
copy_X=True,
n_jobs=1

<\div>

In [33]:
lin_model = LinearRegression()

start = time.time()
lin_model.fit(train_x, train_y)
print("Training took {} seconds".format(round(time.time() - start)))

Training took 11 seconds


### Scoring

In [34]:
start = time.time()
rmsle_cv(lin_model)
print("\nScoring took {} seconds".format(round(time.time() - start)))

[ 0.2640429   0.26175571]

0.262899305367

Scoring took 12 seconds


## Random Forest Regressor

WARNING: Takes a lot of time

In [24]:
rf_model = RandomForestRegressor()

start = time.time()
rf_model.fit(train_x, train_y)
print("Training took {} seconds".format(round(time.time() - start)))

[ 0.27978732  0.28013387  0.28058117]

0.280167454734


### Scoring

In [ ]:
start = time.time()
rmsle_cv(rf_model)
print("\nScoring took {} seconds".format(round(time.time() - start)))

## Logistic Regression

(Training takes 79 seconds)

(Scoring takes 103 seconds)

(open Markdown for notes on [Logistic Regression](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) and parameter search attempts)

<div hidden>

# Logistic Parameters

penalty=’l2’,
dual=False,
tol=0.0001,
C=1.0,
fit_intercept=True,
intercept_scaling=1,
class_weight=None,
random_state=None,
solver=’liblinear’,
max_iter=100,
multi_class=’ovr’,
verbose=0,
warm_start=False,
n_jobs=1

<\div>

In [35]:
log_model = LogisticRegression()

start = time.time()
log_model.fit(train_x, train_y)
print("Training took {} seconds".format(round(time.time() - start)))

Training took 76 seconds


### Scoring

In [36]:
start = time.time()
rmsle_cv(log_model)
print("\nScoring took {} seconds".format(round(time.time() - start)))

[ 0.28573124  0.2826771 ]

0.284204172243

Scoring took 115 seconds


## Lasso Regression

(Training takes 71 seconds)

(Scoring takes 72 seconds)

(open Markdown for notes on [Lasso](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) and parameter search attempts)

<div hidden>

# Lasso Parameters

alpha=1.0,
fit_intercept=True,
normalize=False,
precompute=False,
copy_X=True,
max_iter=1000,
tol=0.0001,
warm_start=False,
positive=False,
random_state=None,
selection=’cyclic’

<\div>

In [37]:
las_model = Lasso(random_state=17)

start = time.time()
las_model.fit(train_x, train_y)
print("Training took {} seconds".format(round(time.time() - start)))

Training took 93 seconds


### Scoring

In [38]:
start = time.time()
rmsle_cv(las_model)
print("\nScoring took {} seconds".format(round(time.time() - start)))

[ 0.27082038  0.26825154]

0.269535959388

Scoring took 76 seconds


## Ridge Regression

(Training takes 5 seconds)

(Scoring takes 5 seconds)


(open Markdown for notes on [Ridge](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html) and parameter search attempts)

<div hidden>

# Ridge Parameters

alpha=1.0,
fit_intercept=True,
normalize=False,
copy_X=True,
max_iter=None,
tol=0.001,
solver=’auto’,
random_state=None

<\div>

In [39]:
rid_model = Ridge(random_state=17)

start = time.time()
rid_model.fit(train_x, train_y)
print("Training took {} seconds".format(round(time.time() - start)))

Training took 6 seconds


### Scoring

In [40]:
start = time.time()
rmsle_cv(rid_model)
print("\nScoring took {} seconds".format(round(time.time() - start)))

[ 0.26399949  0.2616963 ]

0.262847894361

Scoring took 6 seconds


## XGB Regressor

(Training takes 544, 533 seconds)

(Scoring takes 453, 487 seconds)

In [41]:
xgb_model = XGBRegressor()

start = time.time()
xgb_model.fit(train_x, train_y)
print("Training took {} seconds".format(round(time.time() - start)))

Training took 553 seconds


### Scoring

In [42]:
start = time.time()
rmsle_cv(xgb_model)
print("\nScoring took {} seconds".format(round(time.time() - start)))

[ 0.26217886  0.2599844 ]

0.261081632436

Scoring took 487 seconds


### Get predictions

In [43]:
# lgbm_pred = lgbm_model.predict(test_x)
lin_pred  = lin_model.predict(test_x)
# rf_pred   = rf_model.predict(test_x)
log_pred  = log_model.predict(test_x)
las_pred  = las_model.predict(test_x)
rid_pred  = rid_model.predict(test_x)
xgb_pred  = xgb_model.predict(test_x)

### Final predictions

In [44]:
final_pred = 0.2 * lin_pred + 0.1*log_pred + 0.15*las_pred + 0.25*rid_pred + 0.3*xgb_pred

### Restrict predictions to appropriate range

In [45]:
final_pred = np.clip(final_pred, 0, 1)

# Sanity check
any(final_pred < 0) or any(final_pred > 1)

False

### Save file to CSV

In [46]:
pd.DataFrame({
    "SK_ID_CURR": test_id,
    "TARGET": final_pred
}).to_csv("../../submissions/stacked_merge_bur_bal.csv", index=False)